# Epithelium

### graph / dataframe interface


## I/O

- hdf_snapshot


## topology

### graphs

- full graph
- cells graph
- junctions graph

### dataframes

- vertex df
- edge df
- triangles

### actions

- add vertex
- add cell
- add junction
- remove cell
- remove junction




## geometry

- position
- height / area / volume


## dynamics

- energy
- gradient
- optimisation
  + local
  + global


## filters

- cell vertices
- junction edges
- local
- active

## events

- division
- T1
- T3
- apoptosis


In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

import pandas as pd
import graph_tool.all as gt

from leg_joint.epithelium import graph_dataframe_interface as gdf_i
